# Import

In [21]:
import csv
import os
import re
import sys

import numpy as np
import pandas as pd
from nltk import download, word_tokenize
from nltk.corpus import stopwords
from stop_words import get_stop_words
from tqdm import tqdm

from urlextract import URLExtract

## Add configuration file

In [22]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [23]:
from ALL import config 
from util import *

## Set condition

In [24]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [25]:
s3 = S3Manager()

# Read data

In [26]:
df_path = s3.download("DataShaping/TweetFinance/master.csv")

In [27]:
df = pd.read_csv(df_path[0], index_col=0)

In [28]:
labels_path = s3.download("DataShaping/TweetFinance/class.csv")

In [29]:
with open(labels_path[0], mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

# Word tokinize

In [30]:
download('punkt')
download("stopwords")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## delete URL

In [31]:
def format_tweet(tweet):
    # mask web urls
    urls = extractor.find_urls(tweet)
    for url in urls:
        tweet = tweet.replace(url, "{{URL}}")
    # format twitter account
    tweet = re.sub(r"\b(\s*)(@[\S]+)\b", r'\1{\2@}', tweet)
    return tweet

In [32]:
extractor = URLExtract()
df["text"] = df.text.progress_apply(format_tweet)

100%|██████████| 21107/21107 [00:25<00:00, 840.33it/s] 


In [33]:
df["words"] = df.text.progress_apply(word_tokenize)

100%|██████████| 21107/21107 [00:04<00:00, 5003.13it/s]


In [34]:
stop_words_add = ["would", "could", "should"]
stop_char = ["==", "--", "\'s", "''", "n't", "``","..", "...", "....", "'m", "'ve","'re", "'d", "'ll", "", "-+", "+-", "_/", "||", "__", "/|", "//"]
stop_words = set(stopwords.words("english") + get_stop_words("english") + stop_words_add + stop_char)

In [35]:
#     一文字以下の単語とstop_word, stop_charを削除
df["words_nonstop"] = df.words.progress_apply(
    lambda words: [
        word for word in words if word.lower() not in stop_words if len(word)> 1
    ]
)

100%|██████████| 21107/21107 [00:00<00:00, 132514.82it/s]


In [36]:
df.words = df.words.progress_apply(lambda words: " ".join(words))
df.words_nonstop = df.words_nonstop.progress_apply(
    lambda words: " ".join(words)
)

100%|██████████| 21107/21107 [00:00<00:00, 438222.02it/s]


# output

## make file

In [37]:
with open(make_filepath(f"../../temporary/Preprocessing/TweetFinance/class.csv"), "w") as f:
    writer = csv.writer(f)
    writer.writerow(class_labels)
df.to_csv(make_filepath(f"../../temporary/Preprocessing/TweetFinance/master.csv"))

## upload file

In [38]:
s3.upload("../../temporary/Preprocessing/TweetFinance", "Preprocessing/TweetFinance/")

'../../temporary/Preprocessing/TweetFinance'

In [39]:
s3.delete_local_all()

../../temporary/Preprocessing/TweetFinance
/home/jovyan/temporary/DataShaping/TweetFinance/master.csv
/home/jovyan/temporary/DataShaping/TweetFinance/class.csv


In [40]:
shutil.rmtree("../../nltk_data/")